In [ ]:
import os
import sys
import time
import matplotlib.pyplot as plt
import numpy as np
from sklearn.datasets import fetch_openml
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.utils import check_random_state
from sklearn.preprocessing import OneHotEncoder

# Add the parent directory to path to import your modules
os.chdir("../..")
sys.path.append(os.path.dirname(os.path.dirname(os.getcwd())))
from lib import *


In [ ]:
# Authors: The scikit-learn developers
# SPDX-License-Identifier: BSD-3-Clause

# Turn down for faster convergence
t0 = time.time()
train_samples = 5000

# Load data from https://www.openml.org/d/554
X, y = fetch_openml("mnist_784", version=1, return_X_y=True, as_frame=False)

random_state = check_random_state(0)
permutation = random_state.permutation(X.shape[0])
X = X[permutation]
y = y[permutation]
X = X.reshape((X.shape[0], -1))

X_train, X_test, y_train, y_test = train_test_split(
    X, y, train_size=train_samples, test_size=10000
)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Turn up tolerance for faster convergence
clf = LogisticRegression(C=50.0 / train_samples, penalty="l1", solver="saga", tol=0.1)
clf.fit(X_train, y_train)
sparsity = np.mean(clf.coef_ == 0) * 100
score = clf.score(X_test, y_test)
# print('Best C % .4f' % clf.C_)
print("Sparsity with L1 penalty: %.2f%%" % sparsity)
print("Test score with L1 penalty: %.4f" % score)

coef = clf.coef_.copy()
plt.figure(figsize=(10, 5))
scale = np.abs(coef).max()
for i in range(10):
    l1_plot = plt.subplot(2, 5, i + 1)
    l1_plot.imshow(
        coef[i].reshape(28, 28),
        interpolation="nearest",
        cmap=plt.cm.RdBu,
        vmin=-scale,
        vmax=scale,
    )
    l1_plot.set_xticks(())
    l1_plot.set_yticks(())
    l1_plot.set_xlabel("Class %i" % i)
plt.suptitle("Classification vector for...")

run_time = time.time() - t0
print("Example run in %.3f s" % run_time)
plt.show()

In [ ]:
# Convert labels to one-hot encoding for neural network
encoder = OneHotEncoder(sparse_output=False)
y_train_onehot = encoder.fit_transform(y_train.reshape(-1, 1))
y_test_onehot = encoder.transform(y_test.reshape(-1, 1))

# Create validation set
X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train_onehot, test_size=0.2, random_state=42
)

# Print dataset information
print(f"Training set: {X_train.shape[0]} samples")
print(f"Validation set: {X_val.shape[0]} samples")
print(f"Test set: {X_test.shape[0]} samples")
print(f"Input features: {X_train.shape[1]}")
print(f"Output classes: {y_train_onehot.shape[1]}")

### Pengaruh Learning Rate

In [ ]:
# Define depth
depth_variations = [
    [784, 156, 156, 10]
]

# Define activation functions
activation_variations = [
    [ReLU(), ReLU(), Softmax()]
]

In [ ]:
# Create Neural Network
per_sepuluh_network = NeuralNetwork(
    node_counts = depth_variations[0],
    activations = activation_variations[0],
    loss_function = CCE(),
    initialize_methods = NormalInitializer(seed=22)
)

per_seratus_network = NeuralNetwork(
    node_counts = depth_variations[0],
    activations = activation_variations[0],
    loss_function = CCE(),
    initialize_methods = NormalInitializer(seed=22)
)

per_seribu_network = NeuralNetwork(
    node_counts = depth_variations[0],
    activations = activation_variations[0],
    loss_function = CCE(),
    initialize_methods = NormalInitializer(seed=22)
)

In [ ]:
# Create FFNN model
ffnn_per_sepuluh = FFNN(per_sepuluh_network)

ffnn_per_seratus = FFNN(per_seratus_network)

ffnn_per_seribu = FFNN(per_seribu_network)

In [ ]:
# Train the model
per_sepuluh_history = ffnn_per_sepuluh.fit(
    x_train=X_train,
    y_train=y_train,
    batch_size=32,
    epochs=20,
    validation_data=(X_val, y_val),
    learning_rate=0.1,
    verbose=1
)

per_seratus_history = ffnn_per_seratus.fit(
    x_train=X_train,
    y_train=y_train,
    batch_size=32,
    epochs=20,
    validation_data=(X_val, y_val),
    learning_rate=0.01,
    verbose=1
)

per_seribu_history = ffnn_per_seribu.fit(
    x_train=X_train,
    y_train=y_train,
    batch_size=32,
    epochs=20,
    validation_data=(X_val, y_val),
    learning_rate=0.001,
    verbose=1
)

In [ ]:
# Evaluate the model
evaluate_model(ffnn_per_sepuluh, X_test, y_test_onehot)

evaluate_model(ffnn_per_seratus, X_test, y_test_onehot)

evaluate_model(ffnn_per_seribu, X_test, y_test_onehot)

In [ ]:
# Plot the training history
plot_training_loss(per_sepuluh_history, "0.1 LR Training History")

plot_training_loss(per_seratus_history, "0.01 LR Training History")

plot_training_loss(per_seribu_history, "0.001 LR Training History")

In [ ]:
# Plot Weights
ffnn_per_sepuluh.plot_weights(title="FFNN 0.1 LR Weights")

ffnn_per_seratus.plot_weights(title="FFNN 0.01 LR Weights")

ffnn_per_seribu.plot_weights(title="FFNN 0.001 LR Weights")

In [ ]:
# Plot Gradient of Weights
ffnn_per_sepuluh.plot_gradients(title="FFNN 0.1 LR Gradients")

ffnn_per_seratus.plot_gradients(title="FFNN 0.01 LR Gradients")

ffnn_per_seribu.plot_gradients(title="FFNN 0.001 LR Gradients")